In [1]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger('tornado').setLevel(logging.CRITICAL)
logging.getLogger().setLevel(logging.CRITICAL)



from compare_my_stocks.jupyter import get_config
config=get_config()

import json
import pandas
import time


from compare_my_stocks.jupyter.jupytertools import  unite_if_needed,add_change_from,add_target_price_change,highlight_it,calc_closeness, convert_df_dates

from compare_my_stocks.jupyter.jupytertools import load_data
mydata=data=load_data()


In [2]:
translate_dic = {
    'Price':'price',
    'Target Price': 'targetMeanPrice',
    'P/E': 'trailingPE',
    'Forward P/E': 'forwardPE',
    'EPS (ttm)': 'trailingEps',
    'P/S': 'priceToSalesTrailing12Months',
    'P/B': 'priceToBook',
    'Earnings Growth': 'earningsQuarterlyGrowth',
    'EPS next Y': 'forwardEps',
    'Sales Growth': 'revenueGrowth',
    'PEG': 'pegRatio',
    'Perf Year': '52WeekChange',
    'Gross Margin': 'grossMargins',
    'Income': 'netIncomeToCommon',
    'Sales': 'totalRevenue',
    'Market Cap': 'marketCap',
    'Dividend %': 'dividendYield',
}
COLS= translate_dic.keys()


percent_fields = ['Earnings Growth', 'Sales Growth', 'Perf Year', 'Gross Margin', 'Dividend %']


from joblib import Memory
memory = Memory(config.Jupyter.MEMO_FOLDER, verbose=0)

@memory.cache
def get_info(symbolname):
    if len(config.Jupyter.RAPID_YFINANACE_KEY)==0:
        raise "No YFINANCE KEY. Please fill it to get raw data"
        
    import requests

    url = "https://yfinance-stock-market-data.p.rapidapi.com/stock-info"

    payload = { "symbol": symbolname }
    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "X-RapidAPI-Key": config.Jupyter.RAPID_YFINANACE_KEY,
        "X-RapidAPI-Host": config.Jupyter.RAPID_YFINANACE_HOST
    }


    response = requests.post(url, data=payload, headers=headers)
    time.sleep(10)
    return response.json()


def query_symbol(x):
    dic=get_info(x)
    
    dic['data']['price']=(dic['data']['bid']+dic['data']['ask'])/2
    
    
    if 'message' in dic and dic['message']!='Success':
        raise Exception(dic['message'])
        
    return translate_fields_to_dic(translate_dic,percent_fields,dic)
    

def translate_fields_to_dic(translate_dic,percent_fields,dic):
    formatted_dict = {}
    from numerize import numerize
    for k, v in translate_dic.items():
        if dic['data'].get(v):
            if k in percent_fields:
                # multiply by 100 and convert to percent format
                formatted_dict[k] = "{:.2f}%".format(dic['data'].get(v) * 100)
            else:
                # format to two decimal places
                formatted_dict[k] = numerize.numerize(dic['data'].get(v))
        else:
            formatted_dict[k] = None
    return formatted_dict



def process_data(data):
    
    dic={x:unite_if_needed(x,data,query_symbol) for x in data.act.df.columns  }
    
    df=pandas.DataFrame.from_dict(dic)
    df = add_change_from(df,data)
    df = add_change_from(df,data,max=False)
    df = add_target_price_change(df)
    df=df.reindex(COLS)
    
    return df 

df = process_data(data)


#



## Main DF

In [3]:

highlight_it(df)

,AAPL,AMZN,GOOGL,META,MSFT
Price,179.91,138.55,136.38,301.52,335.43
Target Price,187.13,158.44,148.06,338.74,361.02
P/E,30.06,108.02,28.6,35.16,34.6
Forward P/E,29.09,47.03,20.41,19.28,28.43
EPS (ttm),5.97,1.28,4.76,8.54,9.67
P/S,7.31,2.65,5.95,6.41,11.73
P/B,46.59,8.46,6.44,5.82,12.06
Earnings Growth,2.30%,None,14.80%,16.50%,20.00%
EPS next Y,6.17,2.94,6.67,15.57,11.77
Sales Growth,-1.40%,10.80%,7.10%,11.00%,8.30%


## Correlation

In [4]:
mydata.act.df.corr(method='pearson')

Symbols,AAPL,AMZN,GOOGL,META,MSFT
Symbols,,,,,
AAPL,NaN,NaN,NaN,NaN,NaN
AMZN,NaN,1.000000,0.960776,-0.927565,0.925677
GOOGL,NaN,0.960776,1.000000,-0.913161,0.923065
META,NaN,-0.927565,-0.913161,1.000000,-0.956103
MSFT,NaN,0.925677,0.923065,-0.956103,1.000000


## Data

In [5]:
convert_df_dates(mydata.act.df)

Symbols,AAPL,AMZN,GOOGL,META,MSFT
2022-09-11,369.72,-2706.795556,2035.674046,0.00,402.133355
2022-09-12,369.72,-2674.195556,2575.774046,0.00,449.128355
2022-09-13,369.72,-2738.095556,1205.174046,0.00,310.678355
2022-09-14,369.72,-2788.595556,657.374046,0.00,276.488355
2022-09-15,369.72,-2810.795556,644.174046,0.00,209.473355
...,...,...,...,...,...
2023-09-03,369.72,-1085.045556,7899.774046,-1533.10,1788.188355
2023-09-04,369.72,-1085.045556,7899.774046,-1533.10,1788.188355
2023-09-05,369.72,-1153.545556,7694.074046,-1536.70,1813.063355
2023-09-06,369.72,-1232.795556,7618.174046,-1553.05,1862.313355


## Data before processing

In [6]:
convert_df_dates(mydata.afterdata)

Symbols,AAPL,AMZN,GOOGL,META,MSFT
2022-09-11,369.72,-2706.795556,2035.674046,0.00,402.133355
2022-09-12,369.72,-2674.195556,2575.774046,0.00,449.128355
2022-09-13,369.72,-2738.095556,1205.174046,0.00,310.678355
2022-09-14,369.72,-2788.595556,657.374046,0.00,276.488355
2022-09-15,369.72,-2810.795556,644.174046,0.00,209.473355
...,...,...,...,...,...
2023-09-03,369.72,-1085.045556,7899.774046,-1533.10,1788.188355
2023-09-04,369.72,-1085.045556,7899.774046,-1533.10,1788.188355
2023-09-05,369.72,-1153.545556,7694.074046,-1536.70,1813.063355
2023-09-06,369.72,-1232.795556,7618.174046,-1553.05,1862.313355
